<a href="https://www.kaggle.com/code/dzung271828/ouro-trace?scriptVersionId=288567809" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/dzung271828/ouro-trace?scriptVersionId=288557226" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Setup libraries

In [1]:
!uv pip install --upgrade pip
!uv pip uninstall transformers tokenizers accelerate -q

!uv pip install "transformers==4.56.0" "protobuf==5.29.3" -q
!uv pip install torch datasets -q
!uv pip install pandas matplotlib seaborn tqdm wandb pyyaml
!uv pip install bitsandbytes accelerate optimum lm_eval
# !uv pip install -r requirements.txt
!uv pip install --force-reinstall --no-cache-dir "numpy<2.0"

Using Python 3.11.13 environment at: /usr
Resolved 1 package in 41ms                                           
Audited 1 package in 0.17ms
Using Python 3.11.13 environment at: /usr
Audited 6 packages in 105ms
Using Python 3.11.13 environment at: /usr
Resolved 98 packages in 195ms                                        
Installed 1 package in 5ms                                  
 + accelerate==1.12.0
Using Python 3.11.13 environment at: /usr
Resolved 1 package in 146ms                                          
Prepared 1 package in 369ms                                              
Uninstalled 1 package in 30ms
Installed 1 package in 22ms                                 
 ~ numpy==1.26.4


# Suppress warnings

In [2]:
# Suppress warnings for clean output
import warnings
import os

warnings.filterwarnings("ignore", category=UserWarning)
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
print("✅ Packages installed successfully!")

✅ Packages installed successfully!


# Install Libraries

In [3]:
"Built-in libraries"
import re
import sys
import gc
import time
import json
import hashlib
import glob
import zipfile
from io import StringIO
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Any
import yaml
import logging
import random

"Deep learning and NLP libraries"
import torch
import torch.nn.functional as F
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    GenerationConfig,
    logging as hf_logging,
)

"Data processing libraries"
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import wandb
from tqdm.auto import tqdm
from IPython import get_ipython

# Configure logging
logging.getLogger("ContinuousBatchingLogger").setLevel(logging.ERROR)
hf_logging.set_verbosity_error()


print(f"Python Version: {sys.version}")
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
!nvidia-smi

Python Version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
PyTorch Version: 2.6.0+cu124
CUDA Available: True
CUDA Version: 12.4
Fri Dec 26 16:25:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   73C    P8             16W /   70W |       3MiB /  15360MiB |      0%      Default |
|   

In [4]:
import os


def configure_environment_paths():
    """Detect environment and configure paths"""
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"

    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")

    return base_data_path, base_output_path, environment_name


INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()

✅ Environment: Kaggle
📂 Data Path: /kaggle/input/
📦 Output Path: /kaggle/working/


# Setup WANDB

In [5]:
import os
import wandb

if "colab" in ENV_NAME:
    from google.colab import userdata

    try:
        # Ensure 'WANDB_API_KEY' is the exact name in your Colab Secrets (the key icon)
        wandb_key = userdata.get("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Colab Secrets: {e}")

# 2. Check if running in Kaggle
elif "kaggle" in ENV_NAME:
    try:
        from kaggle_secrets import UserSecretsClient

        user_secrets = UserSecretsClient()
        wandb_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Kaggle Secrets: {e}")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dungngocpham171 (dungngocpham171-university-of-science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Config input/output path and clone latest repo

In [6]:
# Clone the latest github repo version
%cd {OUTPUT_PATH}
torch.cuda.empty_cache()
!rm -rf OuroTrace

/kaggle/working


In [7]:
!git clone --branch claude https://github.com/dzungphieuluuky/OuroTrace.git
%cd OuroTrace

Cloning into 'OuroTrace'...
remote: Enumerating objects: 2369, done.
remote: Counting objects: 100% (251/251), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 2369 (delta 147), reused 143 (delta 50), pack-reused 2118 (from 2)
Receiving objects: 100% (2369/2369), 7.09 MiB | 20.68 MiB/s, done.
Resolving deltas: 100% (1504/1504), done.
/kaggle/working/OuroTrace


# Run Benchmark

In [8]:
from src.config_loader import load_config_from_json, post_process_config
from src.new_refactored_runner import run_experiment
from src.evaluation_analysis import analyze_experiment_results

def set_all_seeds(seed):
    random.seed(seed)                          # Python random
    os.environ['PYTHONHASHSEED'] = str(seed)  # Python hash seed
    np.random.seed(seed)                      # NumPy
    torch.manual_seed(seed)                   # PyTorch CPU & GPU

    # Additional GPU-specific settings
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)      # For multi-GPU

set_all_seeds(1415)
# 1. Load Configuration from JSON
config = load_config_from_json("configs/ouro_1.4b_thinking.json")

# 2. Post-process (Convert 'torch.float16' string to object, generate timestamps)
config = post_process_config(config)

config["INFERENCE_STEPS"] = [4]
# config["EVAL_SETTINGS"]["calculate_perplexity"] = False
# config["DATA"]["n_ary"]["num_samples_per_level"] = 0
# config["DATA"]["p_hop"]["num_samples_per_level"] = 0
# config["DATA"]["igsm"]["num_samples"] = 0
# config["DATA"]["reasoning_primitives"]["num_samples"] = 0
# config["ENABLE_HEAVY_BENCHMARKS"] = True
# 4. Execute
print("🚀 Starting Experiment...")
try:
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()
except:
    pass

try:
    simple_reasoning_results, ppl_results, primitives_results, benchmark_results = run_experiment(config)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

🚀 Starting Experiment...
Initializing W&B (timeout: 30s)...


W&B initialized successfully

EXPERIMENT CONFIGURATION
Model Path: ByteDance/Ouro-1.4B-Thinking
UT Steps to Test: [4]
Data Type: torch.bfloat16
4-bit Quantization: False
Torch Compile: False
Max Batch Size: 8
Max New Tokens: 16
Batching: False
Calculate Perplexity: True
Early Exit: 1.0

Quality monitor initialized:
    Garbage threshold: 30%
    Example similarity threshold: 85%
    Min samples before check: 10

LOADING TEST DATASETS
Generating new test datasets...
Generated test datasets successfully

Dataset Summary:
   n_ary       :  500 samples
   p_hop       :  300 samples
   igsm        :  100 samples

Preparing perplexity evaluation data...
Prepared 50 samples for PPL

✅ Configuration saved to ../results_20251226_162535_UT_4/config.json
✅ Task templates saved to ../results_20251226_162535_UT_4/task_templates.json

EXPERIMENT 1/1: UT Steps = 4


LOADING MODEL CONFIGURATION
Model Path: ByteDance/Ouro-1.4B-Thinking
Requested UT Steps: 4
Data Type: torch.bfloat16
4-bit Quantization:

2025-12-26 16:25:37.255227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766766337.271817     336 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766766337.276692     336 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



────────────────────────────────────────────────────────────
APPLYING SAFE OPTIMIZATIONS
────────────────────────────────────────────────────────────
   Flash Attention / SDPA enabled
   TF32 enabled for matmul
   cuDNN auto-tuning enabled
   Memory pool optimized
   Running 3 warmup passes...
   Warmup complete
────────────────────────────────────────────────────────────

MODEL LOADED SUCCESSFULLY
Device: cuda:0
Model dtype: torch.bfloat16
VERIFIED UT steps: 4
VERIFIED early exit: 1.0

Building task templates...
Task templates with pre-tokenized components computed.
    System prompt N_ary tokens: 813 tokens
    System prompt P_hop tokens: 682 tokens
    System prompt IGSM tokens: 1136 tokens
    User prefix tokens: 14 tokens
    User suffix tokens: 6 tokens
    Force start tokens: 4 tokens
Task templates built successfully

✅ Configuration saved to ../results_20251226_162535_UT_4/config.json
✅ Task templates saved to ../results_20251226_162535_UT_4/task_templates.json
Experiment con

Calculating PPL (UT=4):   0%|          | 0/8 [00:00<?, ?it/s]

...

    test_input        full_response  generated_tokens
0  871 + 556 =  [FINAL]  1427 [END]                 8
    test_input        full_response  generated_tokens
0  888 + 759 =  [FINAL]  1647 [END]                 8
    test_input        full_response  generated_tokens
0  670 + 418 =  [FINAL]  1088 [END]                 8
    test_input        full_response  generated_tokens
0  661 + 544 =  [FINAL]  1205 [END]                 8
    test_input        full_response  generated_tokens
0  328 + 902 =  [FINAL]  1230 [END]                 8
    test_input       full_response  generated_tokens
0  696 + 243 =  [FINAL]  939 [END]                 7
    test_input        full_response  generated_tokens
0  810 + 357 =  [FINAL]  1167 [END]                 8
    test_input       full_response  generated_tokens
0  191 + 780 =  [FINAL]  971 [END]                 7
    test_input        full_response  generated_tokens
0  933 + 652 =  [FINAL]  1585 [END]                 8
    test_input        full_respo

   p_hop:   0%|          | 0/300 [00:00<?, ?it/s]

                                          test_input     full_response  \
0  Sequence: B C C B D C B A C A B B B D A A D D ...  [FINAL]  A [END]   

   generated_tokens  
0                 4  
                                          test_input     full_response  \
0  Sequence: A B D C B C C D C D C D D B D B D D ...  [FINAL]  D [END]   

   generated_tokens  
0                 4  
                                          test_input     full_response  \
0  Sequence: C D A D D A B B D B C A D D B C D C ...  [FINAL]  C [END]   

   generated_tokens  
0                 4  
                                          test_input     full_response  \
0  Sequence: B B C C D A C D C D C C C C B B B C ...  [FINAL]  C [END]   

   generated_tokens  
0                 4  
                                          test_input     full_response  \
0  Sequence: C A A C B B D C B C A D A B B D D C ...  [FINAL]  C [END]   

   generated_tokens  
0                 4  
                                   

n_ary/accuracy,▁
n_ary/avg_generation_time,▁
n_ary/avg_tokens,▁
n_ary/num_degenerate,▁
n_ary/num_samples,▁
n_ary/throughput,▁
perplexity,▁
ut_steps,▁▁
val_loss,▁
n_ary/accuracy,0.28
n_ary/avg_generation_time,7.69868


W&B session closed
Cleaning up GPU memory...
GPU memory freed

✅ Periodic save: simple reasoning results to ../results_20251226_162535_UT_4/simple_reasoning.csv
✅ Periodic save: perplexity results to ../results_20251226_162535_UT_4/perplexity.csv
✅ Configuration saved to ../results_20251226_162535_UT_4/config.json
✅ Task templates saved to ../results_20251226_162535_UT_4/task_templates.json


In [9]:
import os
import glob
import zipfile
from typing import List
def find_result_folders(base_path: str) -> List[str]:
    """
    Return a list of absolute paths to all directories under `base_path`
    whose names start with 'results_'.
    """
    pattern = os.path.join(base_path, "results_*")
    # glob returns both files and directories; filter to directories only
    return [p for p in glob.glob(pattern) if os.path.isdir(p)]
def zip_folder(folder_path: str, output_base_path: str) -> bool:
    """
    Zip the contents of `folder_path` into a file named
    <folder_name>.zip` inside `output_base_path`.

    Returns True on success, False otherwise.
    """
    folder_name = os.path.basename(folder_path)
    zip_path = os.path.join(output_base_path, f"{folder_name}.zip")
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(
            zip_path, mode="w", compression=zipfile.ZIP_DEFLATED
        ) as zipf:
            for root, _, files in os.walk(folder_path):
                for file in files:
                    full_path = os.path.join(root, file)
                    # Preserve relative path inside the zip
                    arcname = os.path.relpath(full_path, os.path.dirname(folder_path))
                    zipf.write(full_path, arcname)
        print(f"   ✅ Created ZIP: {os.path.basename(zip_path)}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False
def zip_stats_results_folders(output_base_path: str) -> None:
    """
    Main driver: locate all result folders and zip each one.
    """
    # Ensure the output directory exists
    os.makedirs(output_base_path, exist_ok=True)
    result_folders = find_result_folders(output_base_path)
    if not result_folders:
        print(f"⚠️ No folders starting with 'results_' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = 0
    for folder in result_folders:
        if zip_folder(folder, output_base_path):
            successful += 1
    print(
        f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s)."
    )
if __name__ == "__main__":
    try:
        # Prefer an environment variable; fall back to a global if defined
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        # The script expects a sub‑folder named 'OuroTrace' under OUTPUT_PATH
        target_path = os.path.join(output_root, "")
        zip_stats_results_folders(target_path)
    except Exception as e:
        print(f"❌ An error occurred: {e}")

🔍 Found 3 result folder(s) to zip.
   -> Zipping folder: results_20251226_161418_UT_12...
   ✅ Created ZIP: results_20251226_161418_UT_12.zip
   -> Zipping folder: results_20251226_160923_UT_12...
   ✅ Created ZIP: results_20251226_160923_UT_12.zip
   -> Zipping folder: results_20251226_162535_UT_4...
   ✅ Created ZIP: results_20251226_162535_UT_4.zip

✅ DONE! Successfully zipped 3 out of 3 folder(s).


In [10]:
df_simple = pd.DataFrame(simple_reasoning_results)
df_ppl = pd.DataFrame(ppl_results)

print("Final Inspection:\n")
print("Top 20 Accuracy Report:\n")
print(df_simple.head(20))
print(f"Full Response:\n")
print(df_simple["full_response"])
print("Perplexity Report:\n")
print(df_ppl.head(20))

Final Inspection:

Top 20 Accuracy Report:

   task_type difficulty   test_input expected_answer prediction  is_correct  \
0      n_ary      2_ops  871 + 556 =            1427       1427        True   
1      n_ary      2_ops  888 + 759 =            1647       1647        True   
2      n_ary      2_ops  670 + 418 =            1088       1088        True   
3      n_ary      2_ops  661 + 544 =            1205       1205        True   
4      n_ary      2_ops  328 + 902 =            1230       1230        True   
5      n_ary      2_ops  696 + 243 =             939        939        True   
6      n_ary      2_ops  810 + 357 =            1167       1167        True   
7      n_ary      2_ops  191 + 780 =             971        971        True   
8      n_ary      2_ops  933 + 652 =            1585       1585        True   
9      n_ary      2_ops  995 + 551 =            1546       1546        True   
10     n_ary      2_ops  227 + 885 =            1112       1112        True   
11     n

In [11]:
print(df_simple[["full_response", "generated_tokens"]])

           full_response  generated_tokens
0    [FINAL]  1427 [END]                 8
1    [FINAL]  1647 [END]                 8
2    [FINAL]  1088 [END]                 8
3    [FINAL]  1205 [END]                 8
4    [FINAL]  1230 [END]                 8
..                   ...               ...
604     [FINAL]  C [END]                 4
605     [FINAL]  D [END]                 4
606     [FINAL]  C [END]                 4
607     [FINAL]  C [END]                 4
608     [FINAL]  C [END]                 4

[609 rows x 2 columns]
